## Libraries

In [1]:
import pandas as pd

## Functions

In [2]:
def read_csv(file_name: str) -> pd.DataFrame:
    path = './data/' + file_name + '.csv'

    data = pd.read_csv(path, sep=',')

    return data

### Question 1

In [11]:
SEGMENT = ['Gold', 'Silver', 'Regular']

def q1(data: pd.DataFrame) -> dict:
    d = {
        'regular': data[data['segment'] == SEGMENT[2]].size,
        'silver': data[data['segment'] == SEGMENT[1]].size,
        'gold': data[data['segment'] == SEGMENT[0]].size
    }

    return d


### Question 2

In [48]:
def q2(data: pd.DataFrame):
    cities = {}

    for city in data['province_city']:
        if (cities.get(city) == None):
            cities[city] = 1
        else:
            cities.update({city: cities.get(city)+1})

    return cities

## RUN IT!

In [14]:
cust_data = read_csv('cust')
aum_data = read_csv('aum')
prod_data = read_csv('prod_holding')

### Answer 1

In [20]:
a1 = q1(cust_data)

max_key = max(a1, key=a1.get)
print(max_key, a1[max_key])

regular 24174


### Answer 2

In [57]:
import itertools

a2 = q2(cust_data)

a2_sorted = dict(sorted(a2.items(), key = lambda item: item[1], reverse=True))

dict(itertools.islice(a2_sorted.items(), 5))

{'HA NOI': 3401,
 'HO CHI MINH': 3296,
 'No Info': 456,
 'HAI PHONG': 262,
 'THANH HOA': 179}